In [1]:
test_excel_path = "/home/mann-saradva/Downloads/Hackthon Train Data.xlsx"
LabelEncoder_TERMFREQUENCY_path = "./LabelEncoder_TERMFREQUENCY.obj"
LabelEncoder_SELF_TRADE_path = "./LabelEncoder_SELF_TRADE.obj"
LabelEncoder_SECTOR_path = "./LabelEncoder_SECTOR.obj"
xgboost_trained_model_path = "./xgboost.pkl"

test_set_prediction_excel_path = "./test_set_prediction_excel.xlsx"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_score,recall_score
import pickle

In [3]:
data = pd.read_excel(test_excel_path)

data

,ACCT_NUMBER,BALANCE,SANCTIONAMOUNT,COLLATERALTYPE,COLLATERALVALUE,CREDITLIMIT,DATECLOSED,DATEOPENED,HIGHCREDIT,INSTALLMENTAMOUNT,PASTDUEAMOUNT,REPAYMENTTENURE,TERMFREQUENCY,SELF_TRADE,SECTOR,DOB,Defaulter
0,1024,0.0,50000,NaN,0.0,0.0,2021-09-04,2019-11-29,50000.0,1195.0,0.0,52.0,Bi-weekly,N,NBFC,1971-01-01,0
1,1025,71704.0,80000,NaN,0.0,0.0,NaT,2021-09-04,80000.0,4146.0,0.0,24.0,Monthly,N,NBFC,1971-01-01,0
2,1026,0.0,15000,NaN,NaN,NaN,2014-12-16,2014-01-03,NaN,358.0,0.0,NaN,Monthly,NaN,NaN,1988-05-10,0
3,1027,0.0,20000,NaN,NaN,NaN,2018-02-15,2017-03-15,NaN,907.0,NaN,NaN,Bi-weekly,NaN,NaN,1988-05-10,0
4,1028,21795.0,21795,NaN,NaN,NaN,NaT,2021-08-15,NaN,NaN,0.0,NaN,Monthly,NaN,NaN,1978-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,11218,223852.0,551353,NaN,0.0,0.0,NaT,2020-11-18,551353.0,0.0,223852.0,NaN,NaN,N,BANKS,1989-01-01,1
500,11219,0.0,22000,NaN,NaN,NaN,2019-09-13,2018-06-13,NaN,1730.0,0.0,NaN,Monthly,NaN,NaN,1996-01-01,1
501,11220,6049.0,30000,NaN,NaN,NaN,NaT,2019-02-16,NaN,1650.0,6385.0,NaN,Monthly,NaN,NaN,1996-01-01,1
502,11221,0.0,30000,NaN,NaN,NaN,NaT,2019-06-26,NaN,1096.0,11813.0,NaN,Other,NaN,NaN,1996-01-01,1


In [4]:
data.drop(['HIGHCREDIT'], axis=1,inplace=True)
data.drop(['COLLATERALTYPE'], axis=1,inplace=True)

In [5]:
data['SELF_TRADE'].fillna("Y", inplace=True)
data['DATECLOSED'].fillna(dt.datetime.today().strftime("%m-%d-%Y"), inplace=True)

In [6]:
def find_age_from_DOB(dob):
   
    today = date.today()
   
    age_year=today.year - dob.year
   
    month_difference= abs(dob.month - today.month)
   
    if month_difference > 6:
       
        age_year+=1
       
    return age_year

data['AGE']=data['DOB'].apply(find_age_from_DOB)

data.drop(['DOB'], axis=1,inplace=True)

In [7]:
data['COLLATERALVALUE'] = data['COLLATERALVALUE'].fillna(0)  
data['CREDITLIMIT'] = data['CREDITLIMIT'].fillna(0)  
data['SECTOR'] = data['SECTOR'].fillna("OTHER2")  

In [8]:
def find_days(open_date,close_date):
   
    total_days=[]
    for d1,d2 in zip(close_date,open_date):
       
        total_days.append((d1-d2).days)
       
    return total_days

data['DAYS']=find_days(data['DATEOPENED'],data['DATECLOSED'])

In [9]:
data.drop(['DATEOPENED'], axis=1,inplace=True)
data.drop(['DATECLOSED'], axis=1,inplace=True)

data.drop(['REPAYMENTTENURE'], axis=1,inplace=True)

In [10]:
data['TERMFREQUENCY'] = data['TERMFREQUENCY'].fillna("Monthly")
data['SELF_TRADE'] = data['SELF_TRADE'].fillna("N")
data['SECTOR'] = data['SECTOR'].fillna("BANKS")

data = data.fillna(0) 

In [11]:
LabelEncoder_TERMFREQUENCY = pickle.load(open(LabelEncoder_TERMFREQUENCY_path,'rb'))

data['TERMFREQUENCY'] = LabelEncoder_TERMFREQUENCY.transform(data['TERMFREQUENCY'])

LabelEncoder_SELF_TRADE = pickle.load(open(LabelEncoder_SELF_TRADE_path,'rb'))

data['SELF_TRADE'] = LabelEncoder_SELF_TRADE.transform(data['SELF_TRADE'])

LabelEncoder_SECTOR = pickle.load(open(LabelEncoder_SECTOR_path,'rb'))

data['SECTOR'] = LabelEncoder_SECTOR.transform(data['SECTOR'])

In [12]:
predictors = [x for x in data.columns if x not in ['ACCT_NUMBER','Defaulter']]

x = data[predictors]

In [13]:
xgb_model = pickle.load(open(xgboost_trained_model_path, "rb"))

y_pred = xgb_model.predict(x)

In [14]:
df = pd.read_excel(test_excel_path)

df['PREDICTED_DEFAULTER'] = y_pred

df.to_excel(test_set_prediction_excel_path,index=False)

In [16]:
xgb_model = pickle.load(open(xgboost_trained_model_path, "rb"))

y_pred = xgb_model.predict(x)
y_test  = data['Defaulter']

print("Test Accuracy :",accuracy_score(y_test,y_pred))
print("Recall :",recall_score(y_test,y_pred))
print("Precision :",precision_score(y_test,y_pred))
print("Confusion Matrix :\n",confusion_matrix(y_pred,y_test,labels=[1,0]))

Test Accuracy : 0.9285714285714286
Recall : 0.8731707317073171
Precision : 0.9470899470899471
Confusion Matrix :
 [[179  10]
 [ 26 289]]
